# 🚀 01 - Introdução Prática ao HuggingFace
## ⚡ Otimizado para GPU 2GB | Foco em Português e Jurídico

**Objetivo:** Primeiro contato prático com LLMs

## ⚠️ IMPORTANTE: Limites da Sua GPU
- **VRAM:** 2.1 GB 
- **Modelos que cabem:** ≤ 3B parâmetros (com otimização)
- **Sequência máxima:** ~512 tokens
- **Estratégia:** Usar modelos pequenos OU quantização

---

## 🎯 Objetivos Desta Sessão
1. Carregar modelos em português
2. Aprender quantização básica
3. Fazer primeira aplicação jurídica
4. Identificar próximos passos

In [1]:
# Configuração e diagnóstico
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("🧪 DIAGNÓSTICO DO SISTEMA")
print("="*70)

print(f"🔧 PyTorch: {torch.__version__}")
print(f"🎮 GPU disponível: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"💻 GPU: {gpu_name}")
    print(f"💾 VRAM: {vram_gb:.1f} GB")
    
    # Recomendações baseadas na GPU
    if vram_gb < 3:
        print("\n⚠️  ALERTA: GPU COM MEMÓRIA LIMITADA")
        print("✅ Estratégias que vamos usar:")
        print("   1. Modelos pequenos (≤ 3B parâmetros)")
        print("   2. Quantização 4-bit/8-bit")
        print("   3. Sequências curtas (≤ 512 tokens)")
        print("   4. Batch size = 1")
else:
    print("⚠️  Sem GPU - Usando CPU (será lento)")

print("\n" + "="*70)

🧪 DIAGNÓSTICO DO SISTEMA
🔧 PyTorch: 2.3.1+cu118
🎮 GPU disponível: True
💻 GPU: NVIDIA GeForce 930M
💾 VRAM: 2.1 GB

⚠️  ALERTA: GPU COM MEMÓRIA LIMITADA
✅ Estratégias que vamos usar:
   1. Modelos pequenos (≤ 3B parâmetros)
   2. Quantização 4-bit/8-bit
   3. Sequências curtas (≤ 512 tokens)
   4. Batch size = 1



In [2]:
# O Problema - Modelos em Inglês
print("🎯 DESAFIO: MODELOS EM INGLÊS VS PORTUGUÊS")
print("="*70)

# Teste com GPT-2 (inglês)
print("\n1. Testando GPT-2 (inglês):")
gpt2_generator = pipeline('text-generation', model='gpt2', max_length=50)

prompt_pt = "No Direito brasileiro, o contrato de trabalho"
result_en = gpt2_generator(prompt_pt, num_return_sequences=1)
print(f"Prompt (PT): '{prompt_pt}'")
print(f"Resposta (EN): {result_en[0]['generated_text']}")
print("👉 Problema: Modelo treinado em inglês, responde em inglês!")

🎯 DESAFIO: MODELOS EM INGLÊS VS PORTUGUÊS

1. Testando GPT-2 (inglês):


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt (PT): 'No Direito brasileiro, o contrato de trabalho'
Resposta (EN): No Direito brasileiro, o contrato de trabalho y tambouso, o cambre, o il fasco de germano de de la spicca coma! Pare a una gente
👉 Problema: Modelo treinado em inglês, responde em inglês!


In [3]:
# Solução - Modelos para Português
print("\n" + "="*70)
print("🇧🇷 SOLUÇÃO: MODELOS PARA PORTUGUÊS")
print("="*70)

print("\n🎯 Opções para português (do mais leve ao mais pesado):")

modelos_pt = [
    {
        "nome": "pierreguillou/gpt2-small-portuguese",
        "desc": "GPT-2 pequeno em português",
        "tamanho": "124M parâmetros",
        "recomendacao": "✅ PERFEITO para 2GB"
    },
    {
        "nome": "pierreguillou/gpt2-medium-portuguese", 
        "desc": "GPT-2 médio em português",
        "tamanho": "355M parâmetros",
        "recomendacao": "✅ CABE na 2GB"
    },
    {
        "nome": "neuralmind/bert-base-portuguese-cased",
        "desc": "BERT para português (encoder)",
        "tamanho": "110M parâmetros", 
        "recomendacao": "✅ CABE, mas é BERT (não gera texto)"
    },
    {
        "nome": "microsoft/phi-2",
        "desc": "Phi-2 (2.7B) - Multilíngue",
        "tamanho": "2.7B parâmetros",
        "recomendacao": "⚠️  PRECISA de quantização 4-bit"
    }
]

for i, modelo in enumerate(modelos_pt, 1):
    print(f"\n{i}. {modelo['nome']}")
    print(f"   📝 {modelo['desc']}")
    print(f"   📊 {modelo['tamanho']}")
    print(f"   💡 {modelo['recomendacao']}")


🇧🇷 SOLUÇÃO: MODELOS PARA PORTUGUÊS

🎯 Opções para português (do mais leve ao mais pesado):

1. pierreguillou/gpt2-small-portuguese
   📝 GPT-2 pequeno em português
   📊 124M parâmetros
   💡 ✅ PERFEITO para 2GB

2. pierreguillou/gpt2-medium-portuguese
   📝 GPT-2 médio em português
   📊 355M parâmetros
   💡 ✅ CABE na 2GB

3. neuralmind/bert-base-portuguese-cased
   📝 BERT para português (encoder)
   📊 110M parâmetros
   💡 ✅ CABE, mas é BERT (não gera texto)

4. microsoft/phi-2
   📝 Phi-2 (2.7B) - Multilíngue
   📊 2.7B parâmetros
   💡 ⚠️  PRECISA de quantização 4-bit


In [4]:
# Primeiro Modelo em Português (GPT-2 Small PT)
print("\n" + "="*70)
print("🚀 TESTANDO: GPT-2 Small Português")
print("="*70)

try:
    print("\n🔧 Carregando GPT-2 Small Portuguese...")
    
    # Pipeline simples
    generator_pt = pipeline(
        'text-generation', 
        model='pierreguillou/gpt2-small-portuguese',
        max_length=100,
        device=0 if torch.cuda.is_available() else -1
    )
    
    print("✅ Modelo carregado com sucesso!")
    
    # Testes
    testes = [
        "No Direito brasileiro,",
        "O contrato de trabalho deve",
        "A Constituição Federal estabelece que",
        "O artigo 5º da CLT determina"
    ]
    
    for prompt in testes:
        print(f"\n📝 Prompt: '{prompt}'")
        result = generator_pt(prompt, num_return_sequences=1, max_new_tokens=50)
        print(f"   Resposta: {result[0]['generated_text']}")
        
except Exception as e:
    print(f"❌ Erro ao carregar modelo: {e}")
    print("\n💡 Alternativa: Vamos tentar BERT para análise (não geração):")
    
    # Fallback para BERT
    from transformers import pipeline as pl
    classifier = pl("text-classification", model="neuralmind/bert-base-portuguese-cased")
    print("✅ BERT carregado para classificação")
    print("   Teste: 'Este contrato é válido'")
    result = classifier("Este contrato é válido")
    print(f"   Resultado: {result}")


🚀 TESTANDO: GPT-2 Small Português

🔧 Carregando GPT-2 Small Portuguese...
✅ Modelo carregado com sucesso!

📝 Prompt: 'No Direito brasileiro,'


Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Resposta: No Direito brasileiro, por ele sendo admitido nos cursos de Direito Internacional e na Universidad de Ijuí.

Foi professor catedrático da Faculdade de Direito da Universidade Federal do Rio de Janeiro (UFRJ); de 1991 a 1995 foi coordenador do Departamento de Direito Internacional da Universidad Nacional

📝 Prompt: 'O contrato de trabalho deve'


Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Resposta: O contrato de trabalho deve a ser entregue a uma pessoa contratada de forma definitiva em primeiro lugar, com prazo de um mês ou mais, em casa dos mesmos, até o final do período, podendo também ocorrer em um outro local onde não o mesmo esteja em actividade.



📝 Prompt: 'A Constituição Federal estabelece que'


Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


   Resposta: A Constituição Federal estabelece que:

Para cumprir a preceito fundamental do Estado, as leis emanam dos particulares como forma de manifestação e por eles também podem ser aplicadas nos diversos níveis de governo o qual por suas decisões, incluindo controle das atividades governamentais, controle social que afeta

📝 Prompt: 'O artigo 5º da CLT determina'
   Resposta: O artigo 5º da CLT determina o princípio da igualdade entre os habitantes dos territórios de ambos os países signatários. Segundo o artigo 6º da CLT, a lei brasileira determina o direito de ambos os países signatários a manter suas fronteiras e também a igualdade de direitos aduaneiros e trabalhistas


In [5]:
# Quantização Básica (SALVA-VIDA para 2GB)
print("\n" + "="*70)
print("⚡ QUANTIZAÇÃO: Fazendo modelos grandes caberem na 2GB")
print("="*70)

print("""
O que é quantização?
• Converter pesos de 32-bit → 8-bit ou 4-bit
• Reduz memória em 75% (8-bit) ou 87.5% (4-bit)
• Pouca perda de qualidade para LLMs

PARA SUA GPU 2GB:
• Sem quantização: apenas modelos ≤ 500M
• Com 8-bit: modelos ≤ 2B  
• Com 4-bit: modelos ≤ 3-4B
""")


⚡ QUANTIZAÇÃO: Fazendo modelos grandes caberem na 2GB

O que é quantização?
• Converter pesos de 32-bit → 8-bit ou 4-bit
• Reduz memória em 75% (8-bit) ou 87.5% (4-bit)
• Pouca perda de qualidade para LLMs

PARA SUA GPU 2GB:
• Sem quantização: apenas modelos ≤ 500M
• Com 8-bit: modelos ≤ 2B  
• Com 4-bit: modelos ≤ 3-4B



In [6]:
# Primeiro modelo com quantização (Phi-2 4-bit)
print("\n" + "="*70)
print("🧪 EXPERIMENTO: Phi-2 com 4-bit na GPU 2GB")
print("="*70)

try:
    print("\n🔧 Tentando carregar Phi-2 com 4-bit...")
    
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
    import torch
    
    # Configuração 4-bit
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
    
    # Carregar modelo com quantização
    model_name = "microsoft/phi-2"
    
    print("📥 Baixando modelo (pode demorar alguns minutos)...")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    
    print("✅ Phi-2 carregado com 4-bit!")
    print(f"   Memória usada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    
    # Teste rápido
    prompt = "Explique o que é um contrato de trabalho:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n📝 Prompt: {prompt}")
    print(f"🤖 Resposta: {resposta}")
    
except Exception as e:
    print(f"❌ Erro: {e}")
    print("\n💡 Possíveis soluções:")
    print("1. Instalar bitsandbytes: pip install bitsandbytes")
    print("2. Usar versão mais recente do transformers")
    print("3. Tentar modelo menor primeiro")
    
    # Instalação rápida se necessário
    import sys
    import subprocess
    
    install = input("\n📦 Instalar bitsandbytes agora? (s/n): ")
    if install.lower() == 's':
        print("Instalando bitsandbytes...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "bitsandbytes"])


🧪 EXPERIMENTO: Phi-2 com 4-bit na GPU 2GB

🔧 Tentando carregar Phi-2 com 4-bit...
📥 Baixando modelo (pode demorar alguns minutos)...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

❌ Erro: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

💡 Possíveis soluções:
1. Instalar bitsandbytes: pip install bitsandbytes
2. Usar versão mais recente do transformers
3. Tentar modelo menor primeiro



📦 Instalar bitsandbytes agora? (s/n):  s


Instalando bitsandbytes...


In [7]:
# Aplicação Jurídica Prática
print("\n" + "="*70)
print("⚖️ APLICAÇÃO PRÁTICA: Análise de Cláusula Contratual")
print("="*70)

print("""
Vamos criar uma ferramenta simples para análise jurídica.

Funcionalidade:
1. Extrair termos importantes de cláusulas
2. Classificar tipo de cláusula
3. Sugerir pontos de atenção
""")

# Carregar modelo para português (usando o que funcionou)
try:
    # Tenta carregar GPT-2 PT, se não, usa BERT
    try:
        generator = pipeline(
            'text-generation',
            model='pierreguillou/gpt2-small-portuguese',
            max_length=200,
            device=0
        )
        modelo_tipo = "gerador"
    except:
        from transformers import pipeline as pl
        classifier = pl("text-classification", model="neuralmind/bert-base-portuguese-cased")
        modelo_tipo = "classificador"
    
    print(f"✅ Modelo {modelo_tipo} carregado para análise jurídica")
    
    # Exemplo de cláusula
    clausula = """
    CLÁUSULA QUARTA - DA MULTA. Em caso de descumprimento das obrigações aqui pactuadas, 
    o inadimplente pagará multa correspondente a 10% do valor do contrato, 
    sem prejuízo das perdas e danos.
    """
    
    print(f"\n📄 Cláusula para análise:")
    print(clausula)
    
    if modelo_tipo == "gerador":
        prompt = f"Analise esta cláusula contratual e identifique:\n1. Tipo de cláusula\n2. Elementos principais\n3. Possíveis problemas\n\nCláusula: {clausula}\n\nAnálise:"
        
        resposta = generator(prompt, max_new_tokens=150, num_return_sequences=1)
        print(f"\n🔍 Análise da IA:")
        print(resposta[0]['generated_text'])
    
    else:
        # Análise com classificador
        analises = [
            "Esta cláusula estabelece penalidade por descumprimento",
            "O valor da multa é de 10% do contrato",
            "Há previsão de perdas e danos além da multa"
        ]
        
        print("\n🔍 Classificações:")
        for texto in analises:
            resultado = classifier(texto)
            print(f"   '{texto}' → {resultado[0]['label']} ({resultado[0]['score']:.2%})")
            
except Exception as e:
    print(f"❌ Erro na análise: {e}")


⚖️ APLICAÇÃO PRÁTICA: Análise de Cláusula Contratual

Vamos criar uma ferramenta simples para análise jurídica.

Funcionalidade:
1. Extrair termos importantes de cláusulas
2. Classificar tipo de cláusula
3. Sugerir pontos de atenção



Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✅ Modelo gerador carregado para análise jurídica

📄 Cláusula para análise:

    CLÁUSULA QUARTA - DA MULTA. Em caso de descumprimento das obrigações aqui pactuadas, 
    o inadimplente pagará multa correspondente a 10% do valor do contrato, 
    sem prejuízo das perdas e danos.
    

🔍 Análise da IA:
Analise esta cláusula contratual e identifique:
1. Tipo de cláusula
2. Elementos principais
3. Possíveis problemas

Cláusula: 
    CLÁUSULA QUARTA - DA MULTA. Em caso de descumprimento das obrigações aqui pactuadas, 
    o inadimplente pagará multa correspondente a 10% do valor do contrato, 
    sem prejuízo das perdas e danos.
    

Análise: 
1. Tipo de cláusula contratual 

2. Elementos principais

3. Possíveis problemas

Cláusula: 
1.           IN DIANDAÇÃO QUARTA - DA MULTA. Em caso de descumprimento das obrigações aqui pactuadas, 
   os inadimplentes pagará multa correspondente a 10% do valor do contrato, 
   sem prejuízo das perdas e danos.
               
Análise: 
1.         
2. AR

In [8]:
# Próximos Passos e Resumo
print("\n" + "="*70)
print("📈 RESUMO DO APRENDIZADO E PRÓXIMOS PASSOS")
print("="*70)

print("""
✅ O QUE FUNCIONOU NA SUA GPU 2GB:
1. DistilGPT-2 (inglês) - Funciona, mas não ideal para PT
2. GPT-2 Small Portuguese - Ótimo! Cabe e gera em português
3. BERTimbau - Para análise/classificação
4. Phi-2 com 4-bit - Potencial, precisa de bitsandbytes

⚡ ESTRATÉGIAS EFETIVAS PARA 2GB:
• Modelos ≤ 500M parâmetros (sem quantização)
• Modelos ≤ 2B parâmetros (com 8-bit)
• Modelos ≤ 3-4B parâmetros (com 4-bit)
• Sequências ≤ 512 tokens
• Batch size = 1
""")

# Verificação final
print("\n🔍 STATUS ATUAL DA GPU:")
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    
    print(f"   Memória alocada: {allocated:.2f} GB")
    print(f"   Memória reservada: {reserved:.2f} GB")
    print(f"   Disponível: {(2.1 - allocated):.2f} GB")
    
    if allocated > 1.5:
        print("   ⚠️  Memória quase cheia! Use quantização.")
    else:
        print("   ✅ Memória OK para próximos experimentos.")

print("\n" + "="*70)


📈 RESUMO DO APRENDIZADO E PRÓXIMOS PASSOS

✅ O QUE FUNCIONOU NA SUA GPU 2GB:
1. DistilGPT-2 (inglês) - Funciona, mas não ideal para PT
2. GPT-2 Small Portuguese - Ótimo! Cabe e gera em português
3. BERTimbau - Para análise/classificação
4. Phi-2 com 4-bit - Potencial, precisa de bitsandbytes

⚡ ESTRATÉGIAS EFETIVAS PARA 2GB:
• Modelos ≤ 500M parâmetros (sem quantização)
• Modelos ≤ 2B parâmetros (com 8-bit)
• Modelos ≤ 3-4B parâmetros (com 4-bit)
• Sequências ≤ 512 tokens
• Batch size = 1


🔍 STATUS ATUAL DA GPU:
   Memória alocada: 0.80 GB
   Memória reservada: 0.86 GB
   Disponível: 1.30 GB
   ✅ Memória OK para próximos experimentos.

